<a id='step1'></a>

# 23 - Edge Effects with Hourly Tracking Dictionary

The tracker dictionary is the internal structure of bifacial_radiance to keep track of the multiple Angles or Hours that it's simulating, the rad files, scene, oct files and analysis. Before release v0.4.3, while you could run analysis various times to model different modules in the field and it would save your CSV irradiances, it only kep track of ONE analysis object (the one that was generated last). Given our new performance module calculation functionality, this was updated so that now it keeps track of every module/row analyzed. See example below.

Here we model a central module and an edge module on the center row of an array, to evaluate edge effects.

In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_23')
if not os.path.exists(testfolder):
    os.makedirs(testfolder)
        
print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_23


In [2]:
import bifacial_radiance
import numpy as np
import pprint   
import pandas as pd

<a id='step2'></a>

In [3]:
import bifacial_radiance
import os 

simulationName = 'Tutorial 23'
moduletype = 'test-module'    
albedo = 0.2    
lat = 37.5   
lon = -77.6

#makeModule parameters
modx=1
mody=2
xgap = 0.01
ygap = 0.0
zgap = 0.15
numpanels = 1

# Scene variables
nMods = 20
nRows = 7
hub_height = 1.5 # meters
pitch = 6 # meters  
gcr = mody/pitch

# Traking parameters
cumulativesky = False
limit_angle = 45 # degrees 
angledelta = 0.01 # 
backtrack = True 


torquetube = True
# the Tracker rotates around the panels
tubeParams = {'diameter':0.15,
              'tubetype':'round',
              'material':'black',
              'axisofrotation':False}

startdate = '11_06_11'     
enddate = '11_06_12'
demo = bifacial_radiance.RadianceObj(simulationName, path=testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat,lon) 
metdata = demo.readWeatherFile(epwfile, starttime=startdate, endtime=enddate)  
mymodule = bifacial_radiance.ModuleObj(name=moduletype, x=modx, y=mody, xgap=xgap, ygap=ygap,   
                zgap=zgap, numpanels=numpanels, tubeParams=tubeParams)
sceneDict = {'pitch':pitch,'hub_height':hub_height, 'nMods': nMods, 'nRows': nRows}  
demo.set1axis(limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, cumulativesky = cumulativesky)
demo.gendaylit1axis()
demo.makeScene1axis(module=mymodule,sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis()


path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_23
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/

Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2021-11-06_1100.oct
Created 1axis_2021-11-06_1200.oct


{'2021-11-06_1100': {'surf_azm': 90.0,
  'surf_tilt': 32.06,
  'theta': -32.06,
  'dni': np.int64(203),
  'ghi': np.int64(303),
  'dhi': np.int64(191),
  'temp_air': np.float64(19.4),
  'wind_speed': np.float64(7.2),
  'skyfile': 'skies\\sky2_37.5_-77.33_2021-11-06_1100.rad',
  'scenes': [<class 'bifacial_radiance.main.SceneObj'> : {'gcr': np.float64(0.333333), 'hpc': False, 'module': <class 'bifacial_radiance.module.ModuleObj'> : {'x': 1, 'y': 2, 'z': 0.02, 'modulematerial': 'black', 'scenex': np.float64(1.01), 'sceney': np.float64(2.0), 'scenez': np.float64(0.225), 'numpanels': 1, 'bifi': 1, 'text': '! genbox black test-module 1 2 0.02 | xform -t -0.5 -1.0 0 -a 1 -t 0 2.0 0\r\n! genrev black tube1 t*1.01 0.075 32 | xform -ry 90 -t -0.505 0 -0.22499999999999998', 'modulefile': 'objects\\test-module.rad', 'glass': False, 'glassEdge': 0.01, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.15}, 'modulefile': 'objects\\test-module.rad', 'name': 'Scene0', 'radfiles': 'objects\\1ax

In [4]:
demo.trackerdict.keys()

dict_keys(['2021-11-06_1100', '2021-11-06_1200'])

In [5]:
demo.analysis1axis(modWanted=[1, 10])

Linescan in process: 1axis_2021-11-06_1100_Scene0_Row4_Module1_Front
Linescan in process: 1axis_2021-11-06_1100_Scene0_Row4_Module1_Back
Saved: results\irr_1axis_2021-11-06_1100_Scene0_Row4_Module1.csv
Index: 2021-11-06_1100. Wm2Front: 331.6499111111111. Wm2Back: 50.24847888888889
Linescan in process: 1axis_2021-11-06_1100_Scene0_Row4_Module10_Front
Linescan in process: 1axis_2021-11-06_1100_Scene0_Row4_Module10_Back
Saved: results\irr_1axis_2021-11-06_1100_Scene0_Row4_Module10.csv
Index: 2021-11-06_1100. Wm2Front: 327.00686666666667. Wm2Back: 32.40893888888889
Linescan in process: 1axis_2021-11-06_1200_Scene0_Row4_Module1_Front
Linescan in process: 1axis_2021-11-06_1200_Scene0_Row4_Module1_Back
Saved: results\irr_1axis_2021-11-06_1200_Scene0_Row4_Module1.csv
Index: 2021-11-06_1200. Wm2Front: 196.3093111111111. Wm2Back: 27.20591555555556
Linescan in process: 1axis_2021-11-06_1200_Scene0_Row4_Module10_Front
Linescan in process: 1axis_2021-11-06_1200_Scene0_Row4_Module10_Back
Saved: resu

{'2021-11-06_1100': {'surf_azm': 90.0,
  'surf_tilt': 32.06,
  'theta': -32.06,
  'dni': np.int64(203),
  'ghi': np.int64(303),
  'dhi': np.int64(191),
  'temp_air': np.float64(19.4),
  'wind_speed': np.float64(7.2),
  'skyfile': 'skies\\sky2_37.5_-77.33_2021-11-06_1100.rad',
  'scenes': [<class 'bifacial_radiance.main.SceneObj'> : {'gcr': np.float64(0.333333), 'hpc': False, 'module': <class 'bifacial_radiance.module.ModuleObj'> : {'x': 1, 'y': 2, 'z': 0.02, 'modulematerial': 'black', 'scenex': np.float64(1.01), 'sceney': np.float64(2.0), 'scenez': np.float64(0.225), 'numpanels': 1, 'bifi': 1, 'text': '! genbox black test-module 1 2 0.02 | xform -t -0.5 -1.0 0 -a 1 -t 0 2.0 0\r\n! genrev black tube1 t*1.01 0.075 32 | xform -ry 90 -t -0.505 0 -0.22499999999999998', 'modulefile': 'objects\\test-module.rad', 'glass': False, 'glassEdge': 0.01, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.15}, 'modulefile': 'objects\\test-module.rad', 'name': 'Scene0', 'radfiles': 'objects\\1ax

In [9]:
# Analysis results are now stored in demo.results
demo.results

,timestamp,name,modNum,rowNum,sceneNum,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,backRatio,rearX,rearY,surf_azm,surf_tilt,theta
0,2021-11-06_1100,1axis_2021-11-06_1100_Scene0,1,4,0,"[0.6914, 0.522, 0.3523, 0.1826, 0.01318, -0.15...","[-9.09, -9.09, -9.09, -9.09, -9.09, -9.09, -9....","[1.097, 1.203, 1.309, 1.415, 1.521, 1.627, 1.7...","[1.071, 1.178, 1.283, 1.39, 1.496, 1.602, 1.70...","[a0.3.a0.test-module.6457, a0.3.a0.test-module...","[a0.3.a0.test-module.2310, a0.3.a0.test-module...","[328.9303, 329.6475, 330.3657, 331.4214, 331.9...","[53.14966999999999, 52.59494, 50.04681, 46.840...","[0.161582889801001, 0.15954854943978208, 0.151...","[0.6753, 0.506, 0.3362, 0.1665, -0.00293, -0.1...","[-9.09, -9.09, -9.09, -9.09, -9.09, -9.09, -9....",90.0,32.06,-32.06
1,2021-11-06_1100,1axis_2021-11-06_1100_Scene0,10,4,0,"[0.6914, 0.522, 0.3523, 0.1826, 0.01318, -0.15...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.097, 1.203, 1.309, 1.415, 1.521, 1.627, 1.7...","[1.071, 1.178, 1.283, 1.39, 1.496, 1.602, 1.70...","[a9.3.a0.test-module.6457, a9.3.a0.test-module...","[a9.3.a0.test-module.2310, a9.3.a0.test-module...","[320.5485, 322.0716, 324.7921, 325.9352, 327.0...","[35.57898, 33.59836, 31.185019999999998, 26.62...","[0.11099371547920057, 0.10431921250053561, 0.0...","[0.6753, 0.506, 0.3362, 0.1665, -0.00293, -0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",90.0,32.06,-32.06
2,2021-11-06_1200,1axis_2021-11-06_1200_Scene0,1,4,0,"[0.7935, 0.596, 0.3987, 0.2012, 0.003906, -0.1...","[-9.09, -9.09, -9.09, -9.09, -9.09, -9.09, -9....","[1.3955, 1.428, 1.46, 1.492, 1.524, 1.557, 1.5...","[1.366, 1.398, 1.431, 1.463, 1.495, 1.527, 1.5...","[a0.3.a0.test-module.6457, a0.3.a0.test-module...","[a0.3.a0.test-module.2310, a0.3.a0.test-module...","[196.1987, 196.2184, 196.2383, 196.2582, 196.2...","[29.65619, 29.22897, 27.398890000000005, 24.28...","[0.15115308535130276, 0.14896065322796828, 0.1...","[0.7886, 0.5913, 0.3938, 0.1963, -0.000977, -0...","[-9.09, -9.09, -9.09, -9.09, -9.09, -9.09, -9....",90.0,9.27,-9.27
3,2021-11-06_1200,1axis_2021-11-06_1200_Scene0,10,4,0,"[0.7935, 0.596, 0.3987, 0.2012, 0.003906, -0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.3955, 1.428, 1.46, 1.492, 1.524, 1.557, 1.5...","[1.366, 1.398, 1.431, 1.463, 1.495, 1.527, 1.5...","[a9.3.a0.test-module.6457, a9.3.a0.test-module...","[a9.3.a0.test-module.2310, a9.3.a0.test-module...","[195.7062, 195.7486, 195.79100000000003, 195.8...","[23.14173, 22.39054, 20.48163, 17.66005, 17.21...","[0.11824669710669816, 0.11438357983873275, 0.1...","[0.7886, 0.5913, 0.3938, 0.1963, -0.000977, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",90.0,9.27,-9.27
